Homework 5 - Model Deployment


Question 1: Install uv

pip install uv

In [8]:
%%bash
uv --version

uv 0.9.6


Question 1: Answer is 0.9.6 



Install Scikit-Learn with UV
What's the first hash for Scikit-Learn you get in the lock file?
Include the entire string starting with sha256:, don't include quotes.



In [10]:
# Initialize an empty uv project
!mkdir homework_folder
%cd homework_folder
!uv init

# Install Scikit-Learn version 1.6.1
!uv add scikit-learn==1.6.1

# Check the lock file for the hash
!cat uv.lock | grep -A 10 "scikit-learn" | grep "sha256"

/workspaces/machine-learning-zoomcamp/05-deployment/homework_folder
Initialized project `homework-folder`
Using CPython 3.13.9
Creating virtual environment at: .venv
Resolved 6 packages in 445ms                                         
Prepared 4 packages in 1.09s                                             
░░░░░░░░░░░░░░░░░░░░ [0/5] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 5 packages in 614ms                               
 + joblib==1.5.2
 + numpy==2.3.4
 + scikit-learn==1.6.1
 + scipy==1.16.3
 + threadpoolctl==3.6.0
sdist = { url = "https://files.pythonhosted.org/packages/e8/5d/447af5ea094b9e4c4054f82e223ada074c552335b9b4b2d14bd9b35a67c4/jo

sha256:b4fc2525eca2c69a59260f583c56a7557c6ccdf8deafdba6e060f94c1c59738e

Question 3: Load the Pipeline with pickle , score the record, and then what is probability that this lead will convert


In [11]:
import pickle
import requests

# Download the model
!wget https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin

# Verify checksum
!md5sum pipeline_v1.bin

# Load the model
with open('pipeline_v1.bin', 'rb') as f_in:
    pipeline = pickle.load(f_in)

# Score the record
client = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

# Convert to dictionary format
client_dict = [client]

# Make prediction
prediction = pipeline.predict_proba(client_dict)[0, 1]
print(f"Probability that this lead will convert: {prediction:.3f}")

# Check which option is closest
options = [0.333, 0.533, 0.733, 0.933]
closest = min(options, key=lambda x: abs(x - prediction))
print(f"Closest option: {closest}")

--2025-10-30 03:27:26--  https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving github.com (github.com)... 4.237.22.38
Connecting to github.com (github.com)|4.237.22.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin [following]
--2025-10-30 03:27:27--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1300 (1.3K) [application/octet-stream]
Saving to: ‘pipeline_v1.bin’

pipeline_v1.bin  

/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.6.1 when using version 1.7.0. Th

Probability that this will convert is 0.534.... the closest available option from list is 0.533



Question 4:  FastAPI for webservice, what is probability that client will get a subscription


In [18]:
# Install FastAPI
!uv add fastapi uvicorn

# Create a file for the FastAPI app
app_code = '''
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import uvicorn





# Load the model
with open('pipeline_v1.bin', 'rb') as f_in:
    pipeline = pickle.load(f_in)

#Create FASTAPI app
app = FASTAPI()

class Lead(BaseModel):
    lead_source:str
    number_of_courses_viewed: int
    annual_income: float

    
@app.post('/predict')
def predict(client: dict):
    # Convert the client data to a dictionary
    client_dict = [client]
    
    # Make a prediction
    prediction = pipeline.predict_proba(client_dict)[0, 1]
    
    return {"prediction": prediction}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

with open('app.py', 'w') as f:
    f.write(app_code)

# Run the FastAPI app (in a separate terminal)
# Command: uvicorn app:app --host 0.0.0.0 --port 8000
print("FastAPI application code saved to app.py")
print("To run the service, use: uvicorn app:app --host 0.0.0.0 --port 8000")


Resolved 21 packages in 0.66ms
Audited 19 packages in 0.25ms
FastAPI application code saved to app.py
To run the service, use: uvicorn app:app --host 0.0.0.0 --port 8000


In [23]:
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}
try:
    # make prediction using the same pipeline
    prediction = pipeline.predict_proba([client])[0][1]

    print(f"Client data: {client_data}")
    print(f"Probability of conversion: {probability:.3f}")

    # Check which option is closest
    options = [0.334, 0.534, 0.734, 0.934]
    closest = min(options, key=lambda x: abs(x - prediction))
    print(f"Closest option: {closest}")
except Exception as e:
    print(f"Error making prediction: {e}")
    


Client data: {'lead_source': 'organic_search', 'number_of_courses_viewed': 4, 'annual_income': 80304.0}
Probability of conversion: 0.534
Closest option: 0.534


Answer Question 4:  0.534


Question 5: Install docker, - use it for the next two questions: 
Download the base image agrigorev/zoomcamp-model:2025.  What is the size of base image (45mb, 121mb ,245mb or 330mb)  get information using docker images it will be in the size column.



In [24]:
#  pull the docker image
!docker pull agrigorev/zoomcamp-model:2025

# check the image size
!docker images | grep "zoomcamp-model" 

2025: Pulling from agrigorev/zoomcamp-model

2667830b: Already exists 
1082f6f0: Already exists 
f5177ae2: Already exists 
357cf7a1: Already exists 
2b9cc6ee: Pulling fs layer 
Digest: sha256:14d79fde0bbf078eb18c99c2bd007205917b758ec11060b2994963a1e485c2ae
Status: Downloaded newer image for agrigorev/zoomcamp-model:2025
docker.io/agrigorev/zoomcamp-model:2025
agrigorev/zoomcamp-model   2025      4a9ecc576ae9   8 days ago    121MB


Answer Question 5:  the size is showing as 121mb 

In [31]:
# Create a Dockerfile
print("creating dockerfile")
dockerfile = """
# Install uv
COPY --from=ghcr.io/astral-sh/uv:latest /uv /bin/uv

# Set working directory
WORKDIR /app

# Copy project files
COPY pyproject.toml .
COPY uv.lock .

# Install dependencies
RUN uv sync --frozen

# Copy application files
COPY app.py .

# Expose port
EXPOSE 8000

# Run the application
CMD ["uv", "run", "uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
"""

with open('Dockerfile', 'w') as f:
    f.write(dockerfile)

print("Dockerfile created successfully!")
print()
print("To build and run the Docker container:")
print("1. docker build -t lead-scoring-app .")
print("2. docker run -p 8000:8000 lead-scoring-app")
print()
print("Then test with the same client data from Question 4")
# Build the Docker image



creating dockerfile
Dockerfile created successfully!

To build and run the Docker container:
1. docker build -t lead-scoring-app .
2. docker run -p 8000:8000 lead-scoring-app

Then test with the same client data from Question 4


In [29]:
print('hello world')

hello world


In [41]:
# Note: The Docker container will use pipeline_v2.bin (which is different from pipeline_v1.bin)
# This explains why we might get a different prediction result in Question 6

print("Important Note:")
print("The Docker base image 'agrigorev/zoomcamp-model:2025' contains pipeline_v2.bin")
print("This is a different model than pipeline_v1.bin used in Questions 3 and 4")
print("Therefore, we expect different prediction results for the same input data")
print()

# For testing purposes, let's show what the request would look like
test_request = '''
import requests

url = "http://localhost:8000/predict"
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

response = requests.post(url, json=client)
result = response.json()
print(f"Probability: {result['probability']:.3f}")
'''

print("Test request code:")
print(test_request)

Important Note:
The Docker base image 'agrigorev/zoomcamp-model:2025' contains pipeline_v2.bin
This is a different model than pipeline_v1.bin used in Questions 3 and 4
Therefore, we expect different prediction results for the same input data

Test request code:

import requests

url = "http://localhost:8000/predict"
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

response = requests.post(url, json=client)
result = response.json()
print(f"Probability: {result['probability']:.3f}")



In [38]:
!docker build -t lead-scoring-app

ERROR: docker: 'docker buildx build' requires 1 argument

Usage:  docker buildx build [OPTIONS] PATH | URL | -

Run 'docker buildx build --help' for more information


In [39]:
docker build -t lead-scoring-app

SyntaxError: invalid syntax (1226635584.py, line 1)

In [40]:
!docker build -t lead-scoring-app 


ERROR: docker: 'docker buildx build' requires 1 argument

Usage:  docker buildx build [OPTIONS] PATH | URL | -

Run 'docker buildx build --help' for more information


In [36]:
!docker build -t lead-scoring-app
!docker run -p 8000:8000 lead-scoring-app

ERROR: docker: 'docker buildx build' requires 1 argument

Usage:  docker buildx build [OPTIONS] PATH | URL | -

Run 'docker buildx build --help' for more information
Traceback (most recent call last):
  File "/usr/local/bin/uvicorn", line 8, in <module>
    sys.exit(main())
             ~~~~^^
  File "/usr/local/lib/python3.13/site-packages/click/core.py", line 1462, in __call__
    return self.main(*args, **kwargs)
           ~~~~~~~~~^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.13/site-packages/click/core.py", line 1383, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.13/site-packages/click/core.py", line 1246, in invoke
    return ctx.invoke(self.callback, **ctx.params)
           ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.13/site-packages/click/core.py", line 814, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.13/site-packages/uvicorn/main.py", line 423, in main
    run(
    ~~~^
        app,
        ^^